In [1]:
using System;
using System.Collections.Generic;
using System.IO;
using System.Net.Http;
using System.Net.Http.Headers;
using System.Text;
using System.Threading.Tasks;


In [2]:
var handler = new HttpClientHandler()
            {
                ClientCertificateOptions = ClientCertificateOption.Manual,
                ServerCertificateCustomValidationCallback =
                        (httpRequestMessage, cert, cetChain, policyErrors) => { return true; }
            };

In [3]:
var client = new HttpClient(handler);

In [17]:
var requestBody = @"{
    ""input_data"": {
	    ""input_string"": [
	      ""Instruct: How use secret in dotnet jupyter notebook?\nOutput:""
	    ],
	    ""parameters"": {
	      ""top_p"": 0.1,
	      ""temperature"": 0.1,
	      ""max_new_tokens"": 1000,
	      ""do_sample"": true
	    }
	  }
}";

var content = new StringContent(requestBody);
content.Headers.ContentType = new MediaTypeHeaderValue("application/json");


In [5]:
// Replace this with the primary/secondary key or AMLToken for the endpoint
const string apiKey = "";
if (string.IsNullOrEmpty(apiKey))
{
    throw new Exception("A key should be provided to invoke the endpoint");
}
client.DefaultRequestHeaders.Authorization = new AuthenticationHeaderValue( "Bearer", apiKey);
client.BaseAddress = new Uri("https://ygobert-9486-gyhci.switzerlandnorth.inference.ml.azure.com/score");

// This header will force the request to go to a specific deployment.
// Remove this line to have the request observe the endpoint traffic rules
content.Headers.Add("azureml-model-deployment", "msft-phi-2-11");


In [18]:
// WARNING: The 'await' statement below can result in a deadlock
// if you are calling this code from the UI thread of an ASP.Net application.
// One way to address this would be to call ConfigureAwait(false)
// so that the execution does not attempt to resume on the original context.
// For instance, replace code such as:
//      result = await DoSomeTask()
// with the following:
//      result = await DoSomeTask().ConfigureAwait(false)
HttpResponseMessage response = await client.PostAsync("", content);

if (response.IsSuccessStatusCode)
{
    string result = await response.Content.ReadAsStringAsync();
    Console.WriteLine("Result: {0}", result);
}
else
{
    Console.WriteLine(string.Format("The request failed with status code: {0}", response.StatusCode));

    // Print the headers - they include the requert ID and the timestamp,
    // which are useful for debugging the failure
    Console.WriteLine(response.Headers.ToString());

    string responseContent = await response.Content.ReadAsStringAsync();
    Console.WriteLine(responseContent);
}

Result: [{"0": "Instruct: How use secret in dotnet jupyter notebook?\nOutput: To use a secret in a dotnet jupyter notebook, you can use the following code:\n\n```\nimport jupyter_client\n\n# Create a client object\nclient = jupyter_client.JupyterClient()\n\n# Get the secret\nsecret = client.secrets.get_secret_value('my-secret')\n\n# Print the secret\nprint(secret)\n```\n\nThis code creates a client object that connects to the JupyterHub server and retrieves the secret value with the name'my-secret'. You can replace'my-secret' with the name of your secret.\n"}]
